In [1]:
import ee
#import geetools
import numpy as np
import json
import pandas as pd
import geopandas as gpd
import os
import numpy as np

In [4]:
# # # Trigger the authentication flow.
# ee.Authenticate()

# # # Initialize the library.
ee.Initialize()

In [60]:
L9_images = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
                      .filterDate('1990-01-01', '2024-01-01') # filter by date range
                      .filter(ee.Filter.eq("TARGET_WRS_PATH",140))
                      .filter(ee.Filter.eq("TARGET_WRS_ROW",41))
                      .sort('system:time_start'))

L8_images = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                      .filterDate('1990-01-01', '2024-01-01') # filter by date range
                      .filterDate('2013-10-01', '2013-10-11')
                      .filter(ee.Filter.eq("TARGET_WRS_PATH",140))
                      .filter(ee.Filter.eq("TARGET_WRS_ROW",41))
                      .sort('system:time_start'))

L7_images = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
                      .filterDate('2000-01-01', '2024-01-01') # filter by date range
                      .filter(ee.Filter.eq("WRS_PATH",140))
                      .filter(ee.Filter.eq("WRS_ROW",41))
                      .sort('system:time_start'))

L5_images = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
                      .filterDate('2000-01-01', '2024-01-01') # filter by date range
                      .filter(ee.Filter.eq("WRS_PATH",140))
                      .filter(ee.Filter.eq("WRS_ROW",41))
                      .sort('system:time_start'))

# count number of images
print('L9: ', L9_images.size().getInfo());
print('L8: ', L8_images.size().getInfo());
print('L7: ', L7_images.size().getInfo());
print('L5: ', L5_images.size().getInfo());
# so we should expect each landsat tile to have ~850 images 1990-2023

L9:  44
L8:  1
L7:  415
L5:  156


In [48]:
# load dem that will be used for shadow masks
dem = (ee.ImageCollection('COPERNICUS/DEM/GLO30')
            .select('DEM')
            .filterBounds(L8_images.first().geometry())
            .mosaic())

In [74]:
### now for each, I think we want to ID water, shadows, snow/ice, clouds
# each should be a binary image. 1=present, 0=absent

def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True);

def ID_LS_shadows(image):
        
        # get solar azimuth and zenith   
        azimuth = image.get('SUN_AZIMUTH')
        zenith = image.get('SUN_ELEVATION')
    
        # calculate shadows
        shadow_map = ee.Terrain.hillShadow(**{
          'image': dem,
          'azimuth': azimuth,
          'zenith': zenith,
          'neighborhoodSize': 200,
          'hysteresis': True
        })
        
        # a little convolution to smooth it out
        boxcar = ee.Kernel.square(60, 'meters', True)
        shadow_map = shadow_map.convolve(boxcar).round()
        return shadow_map

def ID_L8_water(image):
    ndwi = applyScaleFactors(image).normalizedDifference(["SR_B3","SR_B5"])
    water = ndwi.gte(0.188).rename('water')
    return water

def ID_LS_clouds(image):

    # grab the QA_pixel band
    qa_pixel_band = image.select('QA_PIXEL')
        
    # create bitmasks for cloud and shadow (bits 3 and 4)
    # bits 1,3,4 indicate clouds/cloud shadows
    dilated_cloud_mask = 1 << 1
    cloud_mask = 1 << 3
    shadow_mask = 1 << 4

    # test each pixel for cloud and shadow
    dilated_clouds = qa_pixel_band.bitwiseAnd(dilated_cloud_mask).neq(0)
    clouds = qa_pixel_band.bitwiseAnd(cloud_mask).neq(0)
    shadows = qa_pixel_band.bitwiseAnd(shadow_mask).neq(0)
    
    # add them all together
    all_clouds = dilated_clouds.max(clouds).max(shadows).rename('cloud')

    return clouds

L8_water = L8_images.map(ID_L8_water)
L8_shadow = L8_images.map(ID_LS_shadows)
L8_cloud = L8_images.map(ID_LS_clouds)
print(L8_water.first().getInfo())
print(L8_shadow.first().getInfo())
print(L8_cloud.first().getInfo())
# print(L8_images.first().getInfo())


{'type': 'Image', 'bands': [{'id': 'water', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'dimensions': [7641, 7791], 'crs': 'EPSG:32645', 'crs_transform': [30, 0, 366885, 0, -30, 3150615]}], 'properties': {'system:index': 'LC08_140041_20131010'}}
{'type': 'Image', 'bands': [{'id': 'shadow', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': 'LC08_140041_20131010'}}
{'type': 'Image', 'bands': [{'id': 'QA_PIXEL', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'dimensions': [7641, 7791], 'crs': 'EPSG:32645', 'crs_transform': [30, 0, 366885, 0, -30, 3150615]}], 'properties': {'system:index': 'LC08_140041_20131010'}}


In [77]:
# export the image to drive
extent = ee.Geometry.Polygon([ee.Geometry.Point([86.72176150806348,27.844906019876465]),
                             ee.Geometry.Point([86.99641971118848,27.844906019876465]),
                             ee.Geometry.Point([86.99641971118848,27.99840413976646]),
                             ee.Geometry.Point([86.72176150806348,27.99840413976646]),
                             ee.Geometry.Point([86.72176150806348,27.844906019876465])])

task = ee.batch.Export.image.toDrive(
    image = L8_cloud.first(),#.clip(extent), #regional_clipped_image,
    region = extent, # region.bounds()
    folder = None,
    scale = 30,
    maxPixels = int(1e13),
    # crs = 'ESRI:102025',
    # crsTransform = [30,0,0,0,-30,0],
    description = "cloud test",
    skipEmptyTiles = True
    )

task.start()
print('Classified image stack export started')#, f"{description}")

Classified image stack export started
